This code uses positive and negative tweets from the NLTK package and accomadating package functions to tokenize, clean, and build a sentiment analysis model from sample tweets. Afterwards this model will be used to analyze samples of tweets that we pull by key word in real time (hopefully).

In [2]:
import nltk
#nltk.download()
#nltk.download('punkt')
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('stopwords')
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import re, string, random
#from nltk import FreqDist, classify, NaiveBayesClassifier 

In [3]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')

In [4]:
print(positive_tweets[0]) #just to show what is being stored in positive/negative/text

#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)


In [5]:
print(text[0])

RT @KirkKus: Indirect cost of the UK being in the EU is estimated to be costing Britain £170 billion per year! #BetterOffOut #UKIP


In [6]:
#Removes hyperlinks and other special symbols then groups elements of tokenized words together using lemmatization 
def cleanNoise(tokenized, stopWords = ()):
    cleaned = []
    for ele, mark in pos_tag(tokenized):
        ele = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', ele)
        ele = re.sub("(@[A-Za-z0-9_]+)","", ele)
        if mark.startswith("NN"):
            grp = 'n'
        elif mark.startswith('VB'):
            grp = 'v'
        else:
            grp = 'a'
            
         #lemmatizes  
        ele = WordNetLemmatizer().lemmatize(ele,grp)
        
        #Checks to make sure ele is not empty, not a repeat, and not punctuation
        if len(ele)>0:
            if ele not in string.punctuation:
                if ele.lower() not in cleaned:
                    cleaned.append(ele.lower())
    return cleaned
        

In [7]:
stopWords = stopwords.words('english')
test = twitter_samples.tokenized('positive_tweets.json')[0]
print(test)
print(cleanNoise(test,stopWords))

['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'being', 'top', 'engaged', 'members', 'in', 'my', 'community', 'this', 'week', ':)']
['#followfriday', 'for', 'be', 'top', 'engage', 'member', 'in', 'my', 'community', 'this', 'week', ':)']


In [8]:
def modelIn(cleanInput):
    for x in cleanInput:
        yield dict([y, True] for y in x)

In [9]:
def modelMaker():
    posTweets = twitter_samples.strings('positive_tweets.json')
    negTweets = twitter_samples.strings('negative_tweets.json')
    text = twitter_samples.strings('tweets.20150430-223406.json')
    stopWords = stopwords.words('english')
    
    posTokens = twitter_samples.tokenized('positive_tweets.json')
    negTokens = twitter_samples.tokenized('negative_tweets.json')
    posCleanTokens = []
    negCleanTokens = []
    
    for x in posTokens:
        clean = cleanNoise(x,stopWords)
        posCleanTokens.append(clean)
    for x in negTokens:
        clean = cleanNoise(x,stopWords)
        negCleanTokens.append(x)
        
    #posTrue = {}
    #negTrue = {}
    #for x in posCleanTokens:
     #   posTrue.append([x, True])
    #for x in negCleanTokens:
     #   negTrue.append([x,True])
    posTrue = modelIn(posCleanTokens)
    negTrue = modelIn(negCleanTokens)
    
    posData = []
    negData = []
    
    for x in posTrue:
        posData.append((x, "Positive"))
    for x in negTrue:
        negData.append((x,"Negative"))
    
    posNegData = posData + negData
    
    #print(posNegData[:5])
    random.shuffle(posNegData)

    train_data = posNegData[:7000]
    test_data = posNegData[7000:]
    
    #right now the model uses Naive bayes but if we are going to do collapsed gibbs we will replace 
    #This part and we will also probably have to reformat the input
    classifier = nltk.NaiveBayesClassifier.train(train_data)
    return(classifier)
    

In [10]:
words = ['be','good','saw']
dict([x,True] for x in words)

{'be': True, 'good': True, 'saw': True}

In [11]:
model = modelMaker()

In [12]:
custom_tweet = "I love sunshine and rainbows"
custom_tokens = cleanNoise(word_tokenize(custom_tweet))
print(custom_tweet, model.classify(dict([token, True] for token in custom_tokens)))

I love sunshine and rainbows Positive


In [13]:
###Using twitter examples

In [14]:
import pandas as pd
import tweepy
from textblob import TextBlob
from gensim import corpora, models
import gensim

In [15]:
auth = tweepy.OAuthHandler('9dQrv1fgVjfRTw4x1vu9Ld1A9', 'ONppfYqFHhqqNaMMWFploNjWCOdXuiOPVwhaiQlH5a6cx7N8Hs')
auth.set_access_token('1381356481161781251-TuZA3s3iF8qHHYI5ikIMlVmRo4grTm', 'u3QMibznnpzi3ikbR6VDssnqc15TCq5f8gyvofPbtExQ7')
api = tweepy.API(auth)

In [16]:
def get_tweets(hashtags, date, numtweets):

    tagged_tweets = tweepy.Cursor(api.search, q=hashtags, lang="en",since=date, tweet_mode="extended").items(numtweets)

    tweets_df = pd.DataFrame(columns=('Date and Time', 'id', 'source', 'Geo', 'Place', 'Retweets', 'tweet', 'Sentiment Polarity'))
    list_tweets = []
    i = 1
    for tweet in tagged_tweets:

        list_tweets.append(tweet.full_text)
        
        tweets_df.at[i, 'id'] = tweet.id
        tweets_df.at[i, 'source'] = tweet.source
        tweets_df.at[i, 'tweet'] = tweet.full_text
        tweets_df.at[i, 'Geo'] = tweet.geo
        tweets_df.at[i, 'Place'] = tweet.place
        tweets_df.at[i, 'Favorites'] = tweet.favorite_count
        tweets_df.at[i, 'Retweets'] = tweet.retweet_count
        tweets_df.at[i, 'Date and Time'] = tweet.created_at
        analysis = TextBlob(tweet.full_text)
        tweets_df.at[i, 'Sentiment Polarity'] = analysis.sentiment.polarity
        i = i + 1

    tweets_df.to_csv(hashtags+'tweets.csv', encoding='utf-8')
    #print(tweets_df.head())
    #print(list_tweets[0])
    return (list_tweets)



In [17]:
tweets = get_tweets('bostoncollege', '2020-04-01', 100)

In [18]:
pos = 0
neg = 0
for x in tweets:
    custom_tokens = cleanNoise(word_tokenize(x))
    res = model.classify(dict([token, True] for token in custom_tokens))
    if res == 'Positive':
        pos+=1
    else:
        neg +=1
print(pos,neg)

94 6


In [19]:
testTweet = tweets[0]
print(testTweet)

BC Baseball News - @BCBirdBall's starting lineup for Game 2 with Pitt. #bcbirdball #bostonathletes #birdball #bostonsports #boston #bceagles #forboston #bostoncollege #bostoncollegeathletics #baseball #collegebaseball #bostonathletemagazine https://t.co/sOTu4xfp2c


In [20]:
custom_tokens = cleanNoise(word_tokenize(testTweet))
print(custom_tokens, model.classify(dict([token, True] for token in custom_tokens)))

['bc', 'baseball', 'news', 'bcbirdball', "'s", 'start', 'lineup', 'for', 'game', '2', 'with', 'pitt', 'bostonathletes', 'birdball', 'bostonsports', 'boston', 'bceagles', 'forboston', 'bostoncollege', 'bostoncollegeathletics', 'collegebaseball', 'bostonathletemagazine', 'http', '//t.co/sotu4xfp2c'] Positive


In [33]:
cleanedTweets = []
for x in tweets: 
    clean = cleanNoise(word_tokenize(x))
    cleanedTweets.append(clean)
    
# for each in cleanedTweets:
#     print(each)
print(cleanedTweets)

[['bc', 'baseball', 'news', 'bcbirdball', "'s", 'start', 'lineup', 'for', 'game', '2', 'with', 'pitt', 'bostonathletes', 'birdball', 'bostonsports', 'boston', 'bceagles', 'forboston', 'bostoncollege', 'bostoncollegeathletics', 'collegebaseball', 'bostonathletemagazine', 'http', '//t.co/sotu4xfp2c'], ['bc', 'baseball', 'score', 'final', 'pitt', '4', 'bcbirdball', '3', 'bostonathletes', 'birdball', 'bostonsports', 'bceagles', 'forboston', 'boston', 'bostoncollege', 'collegebaseball', 'bostonathletemagazine', 'http', '//t.co/lfnfb9lvke'], ['rt', 'hasbcdivested', 'no', 'bostoncollege', 'remain', 'heavily', 'invest', 'in', 'the', 'destructive', 'fossil', 'fuel', 'industry', 'despite', 'pope', "'s", 'repeated', 'call', 't…'], ['rt', 'charitiesboston', 'bostoncollege', 'volunteer', 'load', 'baby', 'supply', 'and', 'food', 'for', 'our', 'community', 'shower', 'with', 'unitedwaymabay', 'staysafe…'], ['bc', 'softball', 'news', 'bc_softball', "'s", 'start', 'lineup', 'for', 'game', '2', 'of', 'to

In [31]:
dictionary = corpora.Dictionary(cleanedTweets)
corpus = [dictionary.doc2bow(text) for text in cleanedTweets]
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 10, id2word = dictionary, passes=100)

print(ldamodel.print_topics())

#Need to remove words that don't mean anything
#of, the, http, a, as, to, rt, have, has, had, us, me, you, what, for

[(0, '0.023*"week" + 0.020*"of" + 0.019*"the" + 0.019*"http" + 0.016*"below" + 0.016*"premium" + 0.016*"another" + 0.016*"details" + 0.016*"//t.co/b3ffdkyaht" + 0.016*"visitor"'), (1, '0.023*"a" + 0.020*"as" + 0.020*"to" + 0.020*"college" + 0.020*"boston" + 0.018*"rt" + 0.015*"coach" + 0.015*"have" + 0.015*"\'s" + 0.015*"see"'), (2, '0.029*"to" + 0.023*"rt" + 0.022*"bostoncollege" + 0.022*"http" + 0.021*"in" + 0.017*"their" + 0.017*"us" + 0.017*"join" + 0.017*"mia" + 0.017*"alumnus"'), (3, '0.017*"bostoncollege" + 0.017*"http" + 0.017*"our" + 0.009*"you" + 0.009*"your" + 0.009*"all" + 0.009*"an" + 0.009*"for" + 0.009*"with" + 0.009*"be"'), (4, '0.032*"rt" + 0.023*"and" + 0.020*"with" + 0.020*"over" + 0.020*"acc" + 0.020*"throw" + 0.020*"second-ever" + 0.020*"myers" + 0.020*"mitch" + 0.020*"pitt_base"'), (5, '0.036*"bostoncollege" + 0.033*"the" + 0.021*"http" + 0.016*"in" + 0.015*"be" + 0.014*"to" + 0.011*"heavily" + 0.011*"destructive" + 0.011*"fossil" + 0.011*"fuel"'), (6, '0.037*"the